## Dynamic Partition Overwrite

In [1]:
# Create Spark Session

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Partition Overwrite") \
    .master("local[2]") \
    .getOrCreate()

spark

In [14]:
# Example dataset
from pyspark.sql.functions import cast, to_date
_data = [
    ["ORD1001", "P003", 70, "01-21-2022"],
    ["ORD1004", "P033", 12, "01-24-2022"],
    ["ORD1005", "P036", 10, "01-20-2022"],
    ["ORD1002", "P016", 2, "01-10-2022"],
    ["ORD1003", "P012", 6, "01-10-2022"],
]

_cols = ["order_id", "prod_id", "qty", "order_date"]

# Create the dataframe
df = spark.createDataFrame(data=_data, schema=_cols)

# Cast the Order date from String to Date
df = df.withColumn("order_date", to_date("order_date" ,"MM-dd-yyyy"))
df.printSchema()
df.show()

root
 |-- order_id: string (nullable = true)
 |-- prod_id: string (nullable = true)
 |-- qty: long (nullable = true)
 |-- order_date: date (nullable = true)

+--------+-------+---+----------+
|order_id|prod_id|qty|order_date|
+--------+-------+---+----------+
| ORD1001|   P003| 70|2022-01-21|
| ORD1004|   P033| 12|2022-01-24|
| ORD1005|   P036| 10|2022-01-20|
| ORD1002|   P016|  2|2022-01-10|
| ORD1003|   P012|  6|2022-01-10|
+--------+-------+---+----------+



In [19]:
# Check the mode for Partition Overwrite
spark.conf.get("spark.sql.sources.partitionOverwriteMode")

'STATIC'

In [17]:
# Lets repartition the data with order_date and write

df.repartition("order_date") \
    .write \
    .format("parquet") \
    .partitionBy("order_date") \
    .mode("overwrite") \
    .save("dataset/orders_partitioned")

In [18]:
%%sh

ls -ltr dataset/orders_partitioned/

total 16
drwxr-sr-x 2 jovyan users 4096 Nov  1 06:03 order_date=2022-01-10
drwxr-sr-x 2 jovyan users 4096 Nov  1 06:03 order_date=2022-01-20
drwxr-sr-x 2 jovyan users 4096 Nov  1 06:03 order_date=2022-01-21
drwxr-sr-x 2 jovyan users 4096 Nov  1 06:03 order_date=2022-01-24
-rw-r--r-- 1 jovyan users    0 Nov  1 06:03 _SUCCESS


In [21]:
# Validate data
from pyspark.sql.functions import count, lit

spark.read.parquet("dataset/orders_partitioned/").groupBy("order_date").agg(count(lit(1))).show()

+----------+--------+
|order_date|count(1)|
+----------+--------+
|2022-01-24|       1|
|2022-01-21|       1|
|2022-01-10|       2|
|2022-01-20|       1|
+----------+--------+



In [22]:
# Lets create our delta dataset for Overwrite

_data = [
    ["ORD1010", "P053", 78, "01-24-2022"],
    ["ORD1011", "P076", 21, "01-20-2022"],
]

_cols = ["order_id", "prod_id", "qty", "order_date"]

# Create the delta dataframe
delta_df = spark.createDataFrame(data=_data, schema=_cols)

# Cast the Order date from String to Date
delta_df = delta_df.withColumn("order_date", to_date("order_date" ,"MM-dd-yyyy"))
delta_df.printSchema()
delta_df.show()

root
 |-- order_id: string (nullable = true)
 |-- prod_id: string (nullable = true)
 |-- qty: long (nullable = true)
 |-- order_date: date (nullable = true)

+--------+-------+---+----------+
|order_id|prod_id|qty|order_date|
+--------+-------+---+----------+
| ORD1010|   P053| 78|2022-01-24|
| ORD1011|   P076| 21|2022-01-20|
+--------+-------+---+----------+



In [23]:
# Lets write to the same location for Orders partitioned

delta_df.repartition("order_date") \
    .write \
    .format("parquet") \
    .partitionBy("order_date") \
    .mode("overwrite") \
    .save("dataset/orders_partitioned")

In [24]:
%%sh

ls -ltr dataset/orders_partitioned/

total 8
drwxr-sr-x 2 jovyan users 4096 Nov  1 06:10 order_date=2022-01-20
drwxr-sr-x 2 jovyan users 4096 Nov  1 06:10 order_date=2022-01-24
-rw-r--r-- 1 jovyan users    0 Nov  1 06:10 _SUCCESS


In [25]:
# Validate data
from pyspark.sql.functions import count, lit

spark.read.parquet("dataset/orders_partitioned/").groupBy("order_date").agg(count(lit(1))).show()

+----------+--------+
|order_date|count(1)|
+----------+--------+
|2022-01-24|       1|
|2022-01-20|       1|
+----------+--------+



### Lets follow the same example but this time with partitionOverwriteMode as "DYNAMIC"

In [28]:
# Setting the partitionOverwriteMode as DYNAMIC

spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
spark.conf.get("spark.sql.sources.partitionOverwriteMode")

'dynamic'

In [29]:
# Example dataset
from pyspark.sql.functions import cast, to_date
_data = [
    ["ORD1001", "P003", 70, "01-21-2022"],
    ["ORD1004", "P033", 12, "01-24-2022"],
    ["ORD1005", "P036", 10, "01-20-2022"],
    ["ORD1002", "P016", 2, "01-10-2022"],
    ["ORD1003", "P012", 6, "01-10-2022"],
]

_cols = ["order_id", "prod_id", "qty", "order_date"]

# Create the dataframe
df = spark.createDataFrame(data=_data, schema=_cols)

# Cast the Order date from String to Date
df = df.withColumn("order_date", to_date("order_date" ,"MM-dd-yyyy"))
df.printSchema()
df.show()

root
 |-- order_id: string (nullable = true)
 |-- prod_id: string (nullable = true)
 |-- qty: long (nullable = true)
 |-- order_date: date (nullable = true)

+--------+-------+---+----------+
|order_id|prod_id|qty|order_date|
+--------+-------+---+----------+
| ORD1001|   P003| 70|2022-01-21|
| ORD1004|   P033| 12|2022-01-24|
| ORD1005|   P036| 10|2022-01-20|
| ORD1002|   P016|  2|2022-01-10|
| ORD1003|   P012|  6|2022-01-10|
+--------+-------+---+----------+



In [30]:
# Lets repartition the data with order_date and write

df.repartition("order_date") \
    .write \
    .format("parquet") \
    .partitionBy("order_date") \
    .mode("overwrite") \
    .save("dataset/orders_partitioned")

In [33]:
%%sh

ls -ltr dataset/orders_partitioned/

total 16
-rw-r--r-- 1 jovyan users    0 Nov  1 06:10 _SUCCESS
drwxr-sr-x 2 jovyan users 4096 Nov  1 06:13 order_date=2022-01-10
drwxr-sr-x 2 jovyan users 4096 Nov  1 06:13 order_date=2022-01-20
drwxr-sr-x 2 jovyan users 4096 Nov  1 06:13 order_date=2022-01-21
drwxr-sr-x 2 jovyan users 4096 Nov  1 06:13 order_date=2022-01-24


In [34]:
# Validate data
from pyspark.sql.functions import count, lit

spark.read.parquet("dataset/orders_partitioned/").groupBy("order_date").agg(count(lit(1))).show()

+----------+--------+
|order_date|count(1)|
+----------+--------+
|2022-01-24|       1|
|2022-01-21|       1|
|2022-01-10|       2|
|2022-01-20|       1|
+----------+--------+



In [39]:
# Lets create our delta dataset for Overwrite

_data = [
    ["ORD1010", "P053", 78, "01-24-2022"],
    ["ORD1011", "P076", 21, "01-10-2022"],
]

_cols = ["order_id", "prod_id", "qty", "order_date"]

# Create the delta dataframe
delta_df = spark.createDataFrame(data=_data, schema=_cols)

# Cast the Order date from String to Date
delta_df = delta_df.withColumn("order_date", to_date("order_date" ,"MM-dd-yyyy"))
delta_df.printSchema()
delta_df.show()

root
 |-- order_id: string (nullable = true)
 |-- prod_id: string (nullable = true)
 |-- qty: long (nullable = true)
 |-- order_date: date (nullable = true)

+--------+-------+---+----------+
|order_id|prod_id|qty|order_date|
+--------+-------+---+----------+
| ORD1010|   P053| 78|2022-01-24|
| ORD1011|   P076| 21|2022-01-10|
+--------+-------+---+----------+



In [40]:
# Lets write to the same location for Orders partitioned

delta_df.repartition("order_date") \
    .write \
    .format("parquet") \
    .partitionBy("order_date") \
    .mode("overwrite") \
    .save("dataset/orders_partitioned")

In [41]:
%%sh

ls -ltr dataset/orders_partitioned/

total 16
-rw-r--r-- 1 jovyan users    0 Nov  1 06:10 _SUCCESS
drwxr-sr-x 2 jovyan users 4096 Nov  1 06:13 order_date=2022-01-21
drwxr-sr-x 2 jovyan users 4096 Nov  1 06:14 order_date=2022-01-20
drwxr-sr-x 2 jovyan users 4096 Nov  1 06:15 order_date=2022-01-10
drwxr-sr-x 2 jovyan users 4096 Nov  1 06:15 order_date=2022-01-24


In [42]:
# Validate data
from pyspark.sql.functions import count, lit

spark.read.parquet("dataset/orders_partitioned/").groupBy("order_date").agg(count(lit(1))).show()

+----------+--------+
|order_date|count(1)|
+----------+--------+
|2022-01-10|       1|
|2022-01-24|       1|
|2022-01-20|       1|
|2022-01-21|       1|
+----------+--------+

